# Распределение встреч между представителями

In [1]:
import pandas as pd
import random
import numpy as np
import time

## Подготовка данных

In [2]:
agents = pd.read_csv('data/F_REAL_WORK.csv', sep=';', index_col='agent_id').to_dict(orient='index')
activities = pd.read_csv('data/F_ACT.csv', sep=';', index_col='ID').to_dict(orient='index')
skills = pd.read_csv('data/F_SKILLS.csv', sep=';')
skills_set = set()
for i in skills.itertuples():
    skills_set.add((i[1], i[2]))
scenarios = pd.read_csv('data/F_SCENARIO.csv', sep=';', index_col='ID').to_dict(orient='index')
distance = pd.read_csv('data/F_DISTANCE.csv', sep=';', index_col=['id1', 'id2'])
id_by_act = {act: i for i, act in enumerate(list(activities))}
distance_by_id = np.zeros((len(activities), len(activities)))

for (i, j), row in distance.iterrows():
    id_i = id_by_act[i]
    id_j = id_by_act[j]
    distance_by_id[id_i][id_j] = row['dist']
    distance_by_id[id_j][id_i] = row['dist']

#### Определение доп.функций

In [44]:
def get_end_by_act(act):
    return activities[act]['DUE_TIME']
def get_distance_between_act(from_act, to_act):
    return distance_by_id[id_by_act[from_act]][id_by_act[to_act]]
def agent_have_skill(agent, skill):
    return (agent, skill) in skills_set
def sort_solution(solution):
    for agent in solution:
        solution[agent].sort(key=get_start_by_act)
def create_empty_solution(agents):
    result = dict()
    score = 0
    for agent in agents:
        result.update({agent: []})
    return result
def get_random_solution():
    solution = create_empty_solution(agents)
    for act in activities:
        skill = activities[act]['SCENARIO_ID']
        random_agent = random.choice(list(skills[skills.SCENARIO_ID == skill]['agent_id']))
        solution[random_agent].append(act)
    sort_solution(solution)
    return solution

#### Подсчет метрик

In [45]:
def compute_total_travel_time(solution):
    total_time = 0
    for agent in solution:
        if len(solution[agent]) == 0:
            continue
        first_act = solution[agent][0]
        total_time += activities[first_act]['from_office']
        for i in range(len(solution[agent]) - 1):
            total_time += get_distance_between_act(solution[agent][i], solution[agent][i+1])
    return total_time
def compute_start_times(solution):
    activity_start = dict()
    for agent in solution:
        if len(solution[agent]) == 0:
            continue
        first_act = solution[agent][0]
        activity_start[first_act] = max(activities[first_act]['READY_TIME'],
                                        agents[agent]['SLOT_START'] + activities[first_act]['from_office'])
        for i in range(1, len(solution[agent])):
            current_act = solution[agent][i]
            prev_act = solution[agent][i - 1]
            dist = get_distance_between_act(prev_act, current_act)
            skill_prev_act = activities[prev_act]['SCENARIO_ID']
            activity_start[current_act] = max(activities[current_act]['READY_TIME'], 
                                              activity_start[prev_act] + scenarios[skill_prev_act]['DURATION'] * 60 + dist)
    return activity_start
def compute_wait_time(solution):
    activity_start = compute_start_times(solution)
    total_wait = 0
    for act, start_act in activity_start.items():
        local_wait = max(0, start_act - activities[act]['DUE_TIME'])
        total_wait += local_wait**2
    return total_wait
def compute_all_metrics(solution):
    total_travel = compute_total_travel_time(solution)
    total_wait = compute_wait_time(solution)
    return total_travel + 0.02 * total_wait

### Эвристика

#### Случайные изменения

 время начала встречи

In [46]:
def get_start_by_act(act):
    return activities[act]['READY_TIME']

определим те же метрики, но для одного агента, чтоб быстрее считать

In [47]:
def compute_total_travel_time_agent(solution, agent):
    total_time = 0    
    if len(solution[agent]) == 0:
        return total_time
    first_act = solution[agent][0]
    total_time += activities[first_act]['from_office']
    for i in range(len(solution[agent]) - 1):
        total_time += get_distance_between_act(solution[agent][i], solution[agent][i+1])
    return total_time
def compute_start_times_agent(solution, agent):
    activity_start = dict()    
    if len(solution[agent]) == 0:
        return activity_start
    first_act = solution[agent][0]
    activity_start[first_act] = max(activities[first_act]['READY_TIME'],
                                    agents[agent]['SLOT_START'] + activities[first_act]['from_office'])
    for i in range(1, len(solution[agent])):
        current_act = solution[agent][i]
        prev_act = solution[agent][i - 1]
        dist = get_distance_between_act(prev_act, current_act)
        skill_prev_act = activities[prev_act]['SCENARIO_ID']
        activity_start[current_act] = max(activities[current_act]['READY_TIME'], 
                                    activity_start[prev_act] + scenarios[skill_prev_act]['DURATION'] * 60 + dist)
    return activity_start
def compute_wait_time_agent(solution, agent):
    activity_start = compute_start_times_agent(solution, agent)
    total_wait = 0
    for act, start_act in activity_start.items():
        local_wait = max(0, start_act - activities[act]['DUE_TIME'])
        total_wait += local_wait**2
    return total_wait
def compute_all_metrics_agent(solution, agent):
    total_travel = compute_total_travel_time_agent(solution, agent)
    total_wait = compute_wait_time_agent(solution, agent)
    return total_travel + 0.02 * total_wait

все же сортируем по началу встречи

при перераспределении задач присваиваем ее агенту, с наименшим показателем приращения метрики

In [48]:
def merge_greedly_by_time(solution, k):
    candidates = random.choices(list(solution), k=k)
    cand_acts = []
    for e in candidates:
        cand_acts.extend(solution[e])
        solution[e] = []
    cand_acts.sort(key=get_start_by_act)
    for act in cand_acts:
        best_candidate = -1
        best_metr = -1
        skill = activities[act]['SCENARIO_ID']
        for candidate in candidates:
            if (not agent_have_skill(candidate, skill)):
                continue
            old_metr=compute_all_metrics_agent(solution, candidate)
            solution[candidate].append(act)
            p_metr = compute_all_metrics_agent(solution, candidate)
            if ((p_metr-old_metr < best_metr) or (best_metr == -1)):
                best_metr = p_metr
                best_candidate = candidate
            solution[candidate].pop()
        assert best_candidate != -1
        solution[best_candidate].append(act)
    return solution

In [49]:
def random_transfer(solution):
    agent = random.choice(list(solution))
    while len(solution[agent]) == 0:
        agent = random.choice(list(solution))
    act = random.choice(tuple(solution[agent]))
    solution[agent].remove(act)
    skill=activities[act]['SCENARIO_ID']
    new_agent = random.choice(list(skills[skills.SCENARIO_ID == skill]['agent_id']))
    solution[new_agent].append(act)

#### создание моего поколения
пытаемся для каждой встречи найти агента, работающего в этот интервал

In [50]:
def get_timestart_solution():
    solution = create_empty_solution(agents)
    for act in activities:
        skill = activities[act]['SCENARIO_ID']
        random_agent = random.choice(list(skills[skills.SCENARIO_ID == skill]['agent_id']))
        for i in range(len(agents)):
            if (agents[random_agent]['SLOT_START']<activities[act]['DUE_TIME'])\
               |(agents[random_agent]['SLOT_END']>activities[act]['READY_TIME']):
                    random_agent = random.choice(list(skills[skills.SCENARIO_ID == skill]['agent_id']))
                    break
            else:
                continue
        solution[random_agent].append(act)
    sort_solution(solution)
    return solution

In [68]:
%%time
generation_size = 1000
generation = []
for _ in range(generation_size):
    one_solution = get_timestart_solution()
    generation.append((compute_all_metrics(one_solution), one_solution))

Wall time: 22min 17s


In [69]:
generation.sort()
generation[0][0]

12736053463.84

#### Получение следующего поколения

Поменяла правила отбора в популяции и добавили другую мутацию

In [71]:
%%time
for i in range(10):
    new_generation = generation[:]
    for g in generation:
        for p in range(10):
            new_solution = merge_greedly_by_time(g[1], 20)
            new_generation.append((compute_all_metrics(new_solution), new_solution))
            new_solution = g[1]
            random_transfer(new_solution)
            new_generation.append((compute_all_metrics(new_solution), new_solution))
    new_generation.sort()
    generation = new_generation[:100]             # 5 лучших
    generation.extend(new_generation[-100:])      # 5 худших
    for k in range(100):                         # 10 случайных
        rand=random.randint(101,len(new_generation)-100)
        generation.extend(new_generation[rand:rand+1])

        
            
    if i % 10 == 9:
        print([s[0] for s in generation][:5])
        

[2672754326.8600001, 2719363712.7800002, 2728187156.2000003, 2729355177.9000001, 2734958073.9000001]
Wall time: 2h 38min


#### Сохранение распределения

In [61]:
def save_solution(file_name, solution):
    data = []
    for agent in solution:
        for act in solution[agent]:
            data.append([agent, act])
    pd.DataFrame(data, columns=['agent_id', 'act_id']).to_csv(file_name, index=False)

In [64]:
save_solution('2783557099.csv', generation[0][1])